In [1]:
import os  
import glob 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import exiftool
import numpy
import re

## ECOWIND-ACCELERATE
# project='ECOWind-accelerate'
# drive ='e:'
# indir = os.path.join(drive,project,'master','files','images')
# save_folder=os.path.join(drive,project,'image-metadata')

## mil
project='Mulroy-Bay'
drive ='e:'
indir = os.path.join(drive,project,'images')
save_folder=os.path.join(drive,project,'image-metadata')


folders=glob.glob(indir+'\*')

for folder in folders:
    # get timestamp of picture folder
    splitname=os.path.split(folder)
    tstamp=splitname[1]
    # build filenam for saving
    savename=os.path.join(save_folder,tstamp+'-coords.csv')
    # check for existence of filename
    if not os.path.exists(savename):
    
        files=glob.glob(folder+'\*.jpg')

        df = pd.DataFrame(columns=['Image_Name','path', 'time','capture time','altitude', 'depth','heading','Lat','latDec','Lon','lonDec','pitch','roll','surge','sway'])

        loopy=range(len(files))

        with exiftool.ExifToolHelper() as et:
                metadata = et.get_metadata(files)

        for i in loopy:
            file=files[i]
            files1 = [file]

            comment=metadata[i]['File:Comment']
            time = re.search('time="(.*)">', comment).group(1)
            capture_time = re.search('<capture_time>(.*)</capture_time>', comment).group(1)
            altitude = re.search('<altitude>(.*)</altitude>', comment).group(1)
            depth = re.search('<depth>(.*)</depth>', comment).group(1)
            heading = re.search('<heading>(.*)</heading>', comment).group(1)
            lat = re.search('<lat>(.*)</lat>', comment).group(1)
            lon = re.search('<lon>(.*)</lon>', comment).group(1)
            pitch = re.search('<pitch>(.*)</pitch>', comment).group(1)
            roll = re.search('<roll>(.*)</roll>', comment).group(1)
            surge = re.search('<surge>(.*)</surge>', comment).group(1)
            sway = re.search('<sway>(.*)</sway>', comment).group(1)


            signlat = 1
            if lat[-1] == "S":
                signlat = -1    
            lenlat = len(lat)
            latCor = signlat * (float(lat[:2]) + float(lat[2:lenlat-2])/60.0)

            signlon=1
            if lon[-1] == "W":
                signlon = -1
            lenlon = len(lon)
            lonCor = signlon * (float(lon[:3]) + float(lon[3:lenlon-2])/60.0)


            basename=os.path.basename(file)
            df.loc[i] = [os.path.basename(file),file,time, capture_time,altitude,depth,heading,lat,latCor,lon,lonCor,pitch,roll,surge,sway]


        df.to_csv(os.path.join(save_folder,tstamp+'-coords.csv'))
        
    else:
        print('file ' +savename +' exists already')

file e:Mulroy-Bay\image-metadata\20210618120309-coords.csv exists already
file e:Mulroy-Bay\image-metadata\20210618120825-coords.csv exists already
file e:Mulroy-Bay\image-metadata\20210618121339-coords.csv exists already
file e:Mulroy-Bay\image-metadata\20210618121647-coords.csv exists already
file e:Mulroy-Bay\image-metadata\20210618121927-coords.csv exists already
file e:Mulroy-Bay\image-metadata\20210618122420-coords.csv exists already
file e:Mulroy-Bay\image-metadata\20210618122505-coords.csv exists already
file e:Mulroy-Bay\image-metadata\20210618122620-coords.csv exists already
file e:Mulroy-Bay\image-metadata\20210618122703-coords.csv exists already
file e:Mulroy-Bay\image-metadata\20210618123241-coords.csv exists already


In [2]:
folder

'e:Mulroy-Bay\\images\\20210621122203'

In [3]:
## count the number of photos
picfolder=os.path.join(drive,project,'image-metadata')

folders=glob.glob(picfolder+'\*')

pics=[]

for folder in folders:
    df=pd.read_csv(folder)
    npics=len(df.index)
    pics.append(npics)

print('Number of photos is ' + str(np.sum(pics)))

Number of photos is 11069


In [4]:
## concatentate the metadata folders
picfolder=os.path.join(drive,project,'image-metadata')

folders=glob.glob(picfolder+'\*')

for x,folder in enumerate(folders):
    if x==1:
        df=pd.read_csv(folder)
    else:
        ds=pd.read_csv(folder)
        df=pd.concat([df,ds])
        
df.to_csv(os.path.join(save_folder,'photo-coords.csv'))


In [5]:
df

,Unnamed: 0,Image_Name,path,time,capture time,altitude,depth,heading,Lat,latDec,Lon,lonDec,pitch,roll,surge,sway
0,0,frame000001_1624018106_084159.jpg,e:Mulroy-Bay\images\20210618120825\frame000001...,2021 06 18 12:08:26.084,1.624018e+09,1.86000,32.78,12.7688,5512.880024N,55.214667,00742.740073W,-7.712334,-0.521851,-2.922540,1.94251,0.019531
1,1,frame000002_1624018106_617530.jpg,e:Mulroy-Bay\images\20210618120825\frame000002...,2021 06 18 12:08:26.617,1.624018e+09,1.86579,32.75,14.4332,5512.880566N,55.214676,00742.739856W,-7.712331,-0.594419,1.206240,1.93652,0.028526
2,2,frame000003_1624018107_150777.jpg,e:Mulroy-Bay\images\20210618120825\frame000003...,2021 06 18 12:08:27.150,1.624018e+09,1.84000,32.73,16.4464,5512.881102N,55.214685,00742.739609W,-7.712327,-0.623572,5.197710,1.93019,0.032227
3,3,frame000004_1624018107_684147.jpg,e:Mulroy-Bay\images\20210618120825\frame000004...,2021 06 18 12:08:27.684,1.624018e+09,1.84000,32.70,18.5419,5512.881626N,55.214694,00742.739346W,-7.712322,-0.384382,8.003480,1.92188,0.030432
4,4,frame000005_1624018108_217538.jpg,e:Mulroy-Bay\images\20210618120825\frame000005...,2021 06 18 12:08:28.217,1.624018e+09,1.86000,32.66,20.5110,5512.882148N,55.214702,00742.739031W,-7.712317,0.031792,10.147900,1.92383,0.026367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,619,frame000620_1624278453_941105.jpg,e:Mulroy-Bay\images\20210621122203\frame000620...,2021 06 21 12:27:33.941,1.624278e+09,3.25523,6.41,291.4560,5511.409554N,55.190159,00746.797039W,-7.779950,-2.821330,-0.732027,1.79700,0.031378
620,620,frame000621_1624278454_474468.jpg,e:Mulroy-Bay\images\20210621122203\frame000621...,2021 06 21 12:27:34.474,1.624278e+09,3.30000,6.42,291.4620,5511.409736N,55.190162,00746.797888W,-7.779965,-2.760870,-1.025750,1.79935,0.032227
621,621,frame000622_1624278455_007821.jpg,e:Mulroy-Bay\images\20210621122203\frame000622...,2021 06 21 12:27:35.007,1.624278e+09,3.25000,6.44,291.4400,5511.409917N,55.190165,00746.798738W,-7.779979,-2.587280,-0.971890,1.80176,0.035156
622,622,frame000623_1624278455_541053.jpg,e:Mulroy-Bay\images\20210621122203\frame000623...,2021 06 21 12:27:35.541,1.624278e+09,3.26137,6.43,291.4740,5511.410093N,55.190168,00746.799574W,-7.779993,-2.749600,-0.214655,1.80260,0.039197
